<a href="https://colab.research.google.com/github/gear-patt/Machine-Learning-Templates/blob/main/LinearRegression_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/KU ML datasets/02-diamonds.csv')

In [ ]:
df

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [ ]:
y = df['price']

In [ ]:
X = df[['carat', 'cut', 'color', 'clarity', 'depth', 'table']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
transformer = make_column_transformer(

    ( OneHotEncoder() , ['cut', 'color', 'clarity']),

    remainder='passthrough' #if we dont put this, it will drop all other columns

)

In [ ]:
X_train_transformed = transformer.fit_transform(X_train)

In [ ]:
X_train_transformed

<43152x23 sparse matrix of type '<class 'numpy.float64'>'
	with 258912 stored elements in Compressed Sparse Row format>

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train_transformed, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([-6.05202355e+02, -4.60799910e+00,  2.68319810e+02,  1.88838459e+02,
        1.52652085e+02,  8.09408435e+02,  6.22357187e+02,  5.15088304e+02,
        3.14246120e+02, -1.64150895e+02, -6.12135119e+02, -1.48481403e+03,
       -3.78753885e+03,  1.58821176e+03, -2.18115122e+02, -1.16970843e+03,
        7.31899790e+02,  4.17077551e+02,  1.27394648e+03,  1.16422683e+03,
        8.90172172e+03, -1.82601148e+01, -2.43635794e+01])

In [ ]:
lr.intercept_

-1168.519631036896

In [ ]:
lr.score(X_train_transformed, y_train)

0.9160324537046203

In [ ]:
y_train.mean()

3926.3548618835744

In [ ]:
error_mean = y_train-y_train.mean()

In [ ]:
MSE_mean = sum(error_mean**2)/len(y_train) #MSE of predicting all outputs by mean

In [ ]:
error_prediction = y_train-lr.predict(X_train_transformed)

In [ ]:
MSE_predict = sum(error_prediction**2)/len(y_train)

In [ ]:
1 - MSE_predict/MSE_mean # R-squared error

0.9160324537046192

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [ ]:
mean_squared_error(y_train, lr.predict(X_train_transformed))

1334342.8800509523

In [ ]:
mean_absolute_error(y_train, lr.predict(X_train_transformed))

802.8635458384751

In [ ]:
mean_absolute_percentage_error(y_train, lr.predict(X_train_transformed))

0.44986866407779225

## Polynomial Features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=4, include_bias=False)

In [ ]:
X_train_transformed.shape

(43152, 23)

In [ ]:
X_train_transformed = poly.fit_transform(X_train_transformed)

In [ ]:
X_train_transformed.shape

(43152, 17549)

As you can see, there are many columns come from PolynomialFeatures.

In [ ]:
lr.fit(X_train_transformed, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([ 0.00469786, -0.00250414,  0.00011111, ...,  0.05184989,
       -0.03943166,  0.01136327])

In [ ]:
lr.coef_

array([ 0.00469786, -0.00250414,  0.00011111, ...,  0.05184989,
       -0.03943166,  0.01136327])

In [ ]:
lr.score(X_train_transformed, y_train)

0.9764733646515255

In [ ]:
X_test_transformed = poly.transform(transformer.transform(X_test)) #One-hot-encoding and polynomial transformation

In [ ]:
y_test_predicted = lr.predict(X_test_transformed)

In [ ]:
mean_squared_error(y_test, y_test_predicted)

417194.7812847618

In [ ]:
lr.score(X_test_transformed, y_test) #R-squared is not commonly used with test set.

0.9739434769686249

## Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
sgdr = SGDRegressor(max_iter=1000, tol=0.001, n_iter_no_change=5, penalty=None)

In [ ]:
sgdr.fit(X_train_transformed, y_train)

SGDRegressor(penalty=None)

In [ ]:
sgdr.score(X_train_transformed, y_train) 

-6.4539761055594175e+37

This means that the model is worse than mean prediction.

In [ ]:
mean_squared_error(sgdr.predict(X_train_transformed), y_train)

1.0256125663333858e+45

In [ ]:
sgdr = SGDRegressor(max_iter=5000, tol=0.0001, n_iter_no_change=100, penalty=None)

In [ ]:
sgdr.fit(X_train_transformed, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


SGDRegressor(max_iter=5000, n_iter_no_change=100, penalty=None, tol=0.0001)

In [ ]:
sgdr.score(X_train_transformed, y_train)

-2.9662098451955686e+37

In [ ]:
mean_squared_error(sgdr.predict(X_train_transformed), y_train)

4.713655647088412e+44